# Evaluation and Results
- Evaluation of each of the augmented datasets
- Results are compared and contrasted with those of the unaltered dataset

In [131]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_fscore_support as score

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from random import randrange
import rocket
import mpdatasets
import augmentation as aug
from dtw import *

<br><br>
 ### Load the original datasets


In [132]:
train_x, train_y, test_x, test_y = mpdatasets.load_datasets()

<br><br>
### Generate the augmented datasets using each of the augmentation methods
- Generate each of the augmented datasets
- The runtime for this is also calculated

In [133]:
rev_tic = time.perf_counter()
reversed_train_x = aug.reverse_dataframe(train_x)

print(reversed_train_x.shape)

reversed_train_x = pd.concat([train_x, reversed_train_x], axis=0)
reversed_train_y = np.append(train_y, train_y, axis=0)
reversed_test_x = test_x
reversed_test_y = test_y

print(f"Reverse Runtime: {time.perf_counter() - rev_tic}")

(1426, 36)
Reverse Runtime: 16.774161499997717


In [134]:
window_size = len(train_x.iloc[0][0])//10
multiplier = 3
ww_tic = time.perf_counter()
warped_train_x = aug.window_warp(train_x, window_size, multiplier)
padded_train_x = aug.pad_series(train_x, window_size, multiplier)

warped_train_x = pd.concat([padded_train_x, warped_train_x], axis=0)
warped_train_y = np.append(train_y, train_y, axis=0)
warped_test_x = aug.pad_series(test_x, window_size, multiplier)
warped_test_y = test_y

print(f"Window Warp Runtime: {time.perf_counter() - ww_tic}")

pd_tic = time.perf_counter()
padded_train_x = aug.pad_series(train_x, window_size, multiplier)
padded_train_x = pd.concat([padded_train_x, padded_train_x], axis=0, ignore_index=True)
padded_train_y = np.append(train_y, train_y, axis=0)
padded_test_x = aug.pad_series(test_x, window_size, multiplier)
padded_test_y = test_y

print(f"Padded Runtime: {time.perf_counter() - pd_tic}")

Window Warp Runtime: 1202.6121115999995
Padded Runtime: 57.2326752999943


In [135]:
dtw_tic = time.perf_counter()

dtw_train_x, dtw_train_y = aug.dtw_interpolate(train_x, train_y)
dtw_test_x = test_x
dtw_test_y = test_y
dtw_train_x = pd.concat([train_x, dtw_train_x], axis=0, ignore_index=True)
dtw_train_y = np.append(train_y, dtw_train_y, axis=0)

print(f"DTW-Interpolation Runtime: {time.perf_counter() - dtw_tic}")


DTW-Interpolation Runtime: 1481.3587011000054


#### Print the size of each dataset produced

In [7]:
print(f"Original: {train_x.shape}\nReversed: {reversed_train_x.shape}\nWarped: {warped_train_x.shape}")
print(f"Padded: {padded_train_x.shape}\nDTW: {dtw_train_x.shape}")

Original: (1426, 36)
Reversed: (2852, 36)
Warped: (2852, 36)
Padded: (2852, 36)
DTW: (2848, 36)


<br><br>
### Run ROCKET on each of the datasets
- Rocket is ran using each dataset as training samples
- The time taken to train ROCKET is also calculated here

In [146]:
og_tic = time.perf_counter()
original_accuracy, original_preds = rocket.run_classifier(train_x, train_y, test_x, test_y)
print(f"Runtime: {time.perf_counter() - og_tic}")

[Pipeline] ............ (step 1 of 3) Processing rocket, total= 2.0min
[Pipeline] ......... (step 2 of 3) Processing normalise, total=   0.9s
[Pipeline] ............. (step 3 of 3) Processing model, total=   1.9s
Runtime: 173.24744140000257


In [138]:
rev_tic = time.perf_counter()
reversed_accuracy, reversed_preds = rocket.run_classifier(reversed_train_x, reversed_train_y, reversed_test_x, reversed_test_y)
print(f"Runtime: {time.perf_counter() - rev_tic}")

[Pipeline] ............ (step 1 of 3) Processing rocket, total= 4.0min
[Pipeline] ......... (step 2 of 3) Processing normalise, total=   1.4s
[Pipeline] ............. (step 3 of 3) Processing model, total=  11.7s
Runtime: 306.3563789000036


In [139]:
ww_tic = time.perf_counter()
warped_accuracy, warped_preds = rocket.run_classifier(warped_train_x, warped_train_y, warped_test_x, warped_test_y)
print(f"Runtime: {time.perf_counter() - ww_tic}")

[Pipeline] ............ (step 1 of 3) Processing rocket, total= 4.8min
[Pipeline] ......... (step 2 of 3) Processing normalise, total=   1.3s
[Pipeline] ............. (step 3 of 3) Processing model, total=   9.6s
Runtime: 362.65380680000817


In [140]:
pd_tic = time.perf_counter()
padded_accuracy, padded_preds = rocket.run_classifier(padded_train_x, padded_train_y, padded_test_x, padded_test_y)
print(f"Runtime: {time.perf_counter() - pd_tic}")

[Pipeline] ............ (step 1 of 3) Processing rocket, total= 4.8min
[Pipeline] ......... (step 2 of 3) Processing normalise, total=   1.3s
[Pipeline] ............. (step 3 of 3) Processing model, total=   8.8s
Runtime: 362.06926669999666


In [141]:
dtw_tic = time.perf_counter()
dtw_accuracy, dtw_preds = rocket.run_classifier(dtw_train_x, dtw_train_y, dtw_test_x, dtw_test_y)
print(f"Runtime: {time.perf_counter() - dtw_tic}")

[Pipeline] ............ (step 1 of 3) Processing rocket, total= 4.0min
[Pipeline] ......... (step 2 of 3) Processing normalise, total=   1.3s
[Pipeline] ............. (step 3 of 3) Processing model, total=   9.4s
Runtime: 303.6038176000002


<br><br>
### Display accuracy of each dataset

#### Original: 
85.21008403361344

#### Reversed:
82.18487394957982

#### Warped:
85.71428571428571

#### Padded:
86.05042016806723

#### DTW:
86.38655462184875


In [15]:
print(f"Original: {original_accuracy}\nReversed: {reversed_accuracy}\nWarped: {warped_accuracy}")
print(f"Padded: {padded_accuracy}\nDTW: {dtw_accuracy}")

Original: 85.21008403361344
Reversed: 82.18487394957982
Warped: 85.71428571428571
Padded: 86.05042016806723
DTW: 86.38655462184875


<br><br>
### Compute Precision, Recall, and F1-Score for each of the datasets
- These three performance metrics are caluclated for each of the datasets

In [58]:
original_precision, original_recall, original_f1, original_support = score(test_y, original_preds, average='macro')

#### Original

Precision: 0.8547047517371634

Recall: 0.8518778801843319

F1-Score: 0.8520355370287889


In [59]:
print("Original")
print(f"Precision: {original_precision}")
print(f"Recall: {original_recall}")
print(f"F1-Score: {original_f1}")

Original
Precision: 0.8547047517371634
Recall: 0.8518778801843319
F1-Score: 0.8520355370287889


In [44]:
reversed_precision, reversed_recall, reversed_f1, reversed_support = score(test_y, reversed_preds, average='macro')

#### Reversed

Precision: 0.8212856881883188

Recall: 0.8207642089093703

F1-Score: 0.8198574065174149


In [45]:
print("Reversed")
print(f"Precision: {reversed_precision}")
print(f"Recall: {reversed_recall}")
print(f"F1-Score: {reversed_f1}")

Reversed
Precision: 0.8212856881883188
Recall: 0.8207642089093703
F1-Score: 0.8198574065174149


In [46]:
warped_precision, warped_recall, warped_f1, warped_support = score(test_y, warped_preds, average='macro')

#### Window Warp

Precision: 0.8610955350149521

Recall: 0.8570199692780338

F1-Score: 0.8568717867383532

In [47]:
print("Window Warp")
print(f"Precision: {warped_precision}")
print(f"Recall: {warped_recall}")
print(f"F1-Score: {warped_f1}")

Window Warp
Precision: 0.8610955350149521
Recall: 0.8570199692780338
F1-Score: 0.8568717867383532


In [48]:
padded_precision, padded_recall, padded_f1, padded_support = score(test_y, padded_preds, average='macro')

#### Padded

Precision: 0.8639138340309427

Recall: 0.8604723502304148

F1-Score: 0.8600626836056275


In [49]:
print("Padded")
print(f"Precision: {padded_precision}")
print(f"Recall: {padded_recall}")
print(f"F1-Score: {padded_f1}")

Padded
Precision: 0.8639138340309427
Recall: 0.8604723502304148
F1-Score: 0.8600626836056275


In [86]:
dtw_precision, dtw_recall, dtw_f1, dtw_support = score(test_y, dtw_preds, average='macro')

#### DTW-Interpolation

Precision: 0.8682049923795176

Recall: 0.863437019969278

F1-Score: 0.8646906071276432

In [87]:
print("DTW-Interpolation")
print(f"Precision: {dtw_precision}")
print(f"Recall: {dtw_recall}")
print(f"F1-Score: {dtw_f1}")

DTW-Interpolation
Precision: 0.8682049923795176
Recall: 0.863437019969278
F1-Score: 0.8646906071276432


<br><br>
### Compute Precision, Recall, and F1-Score by class label
- The same metrics are then calculated per class label
- Values are given in arrays with values in the order of {'Normal', 'Asymmetrical', 'Arch', 'Reduced Range'}

In [72]:
class_labels = ["n","a", "arch", "r"]
original_class_precision, original_class_recall, original_class_f1, original_class_support = score(test_y, original_preds, labels=class_labels)

#### Original

Precision: [0.8028169  0.97058824 0.81208054 0.83333333]

Recall: [0.76       0.88       0.86428571 0.90322581]

F1-Score: [0.78082192 0.92307692 0.83737024 0.86687307]

In [73]:
print("Original")
print(f"Precision: {original_class_precision}")
print(f"Recall: {original_class_recall}")
print(f"F1-Score: {original_class_f1}")

Original
Precision: [0.8028169  0.97058824 0.81208054 0.83333333]
Recall: [0.76       0.88       0.86428571 0.90322581]
F1-Score: [0.78082192 0.92307692 0.83737024 0.86687307]


In [74]:
reversed_class_precision, reversed_class_recall, reversed_class_f1, reversed_class_support = score(test_y, reversed_preds, labels=class_labels)

In [70]:
original_class_precision[1]

0.9705882352941176

#### Reversed

Precision: [0.76811594 0.90714286 0.76870748 0.84117647]

Recall: [0.70666667 0.84666667 0.80714286 0.92258065]

F1-Score: [0.73611111 0.87586207 0.78745645 0.88      ]

In [75]:
print("Reversed")
print(f"Precision: {reversed_class_precision}")
print(f"Recall: {reversed_class_recall}")
print(f"F1-Score: {reversed_class_f1}")

Reversed
Precision: [0.76811594 0.90714286 0.76870748 0.84117647]
Recall: [0.70666667 0.84666667 0.80714286 0.92258065]
F1-Score: [0.73611111 0.87586207 0.78745645 0.88      ]


In [76]:
warped_class_precision, warped_class_recall, warped_class_f1, warped_class_support = score(test_y, warped_preds, labels=class_labels)

#### Window Warp

Precision: [0.80434783 0.98496241 0.83221477 0.82285714]

Recall: [0.74       0.87333333 0.88571429 0.92903226]

F1-Score: [0.77083333 0.92579505 0.85813149 0.87272727]


In [84]:
print("Window Warp")
print(f"Precision: {warped_class_precision}")
print(f"Recall: {warped_class_recall}")
print(f"F1-Score: {warped_class_f1}")

Window Warp
Precision: [0.80434783 0.98496241 0.83221477 0.82285714]
Recall: [0.74       0.87333333 0.88571429 0.92903226]
F1-Score: [0.77083333 0.92579505 0.85813149 0.87272727]


In [80]:
padded_class_precision, padded_class_recall, padded_class_f1, padded_class_support = score(test_y, padded_preds, labels=class_labels)

#### Padded

Precision: [0.8115942  0.98496241 0.81699346 0.84210526]

Recall: [0.74666667 0.87333333 0.89285714 0.92903226]

F1-Score: [0.77777778 0.92579505 0.85324232 0.88343558]


In [85]:
print("Padded")
print(f"Precision: {padded_class_precision}")
print(f"Recall: {padded_class_recall}")
print(f"F1-Score: {padded_class_f1}")

Padded
Precision: [0.8115942  0.98496241 0.81699346 0.84210526]
Recall: [0.74666667 0.87333333 0.89285714 0.92903226]
F1-Score: [0.77777778 0.92579505 0.85324232 0.88343558]


In [82]:
dtw_class_precision, dtw_class_recall, dtw_class_f1, dtw_class_support = score(test_y, dtw_preds, labels=class_labels)

#### DTW-Interpolation

Precision: [0.79194631 0.96376812 0.8962963  0.82080925]

Recall: [0.78666667 0.88666667 0.86428571 0.91612903]

F1-Score: [0.78929766 0.92361111 0.88       0.86585366]


In [88]:
print("DTW-Interpolation")
print(f"Precision: {dtw_class_precision}")
print(f"Recall: {dtw_class_recall}")
print(f"F1-Score: {dtw_class_f1}")

DTW-Interpolation
Precision: [0.79194631 0.96376812 0.8962963  0.82080925]
Recall: [0.78666667 0.88666667 0.86428571 0.91612903]
F1-Score: [0.78929766 0.92361111 0.88       0.86585366]


<br><br>
### Combining augmented training sets for training classifier
- The different permutations of dataset combinations are also examined
- After being joined, each combination is used on ROCKET
- The accuracy achieved and time taken to train the model are both then output

#### DTW and Window Warping

In [111]:
padded_dtw = aug.pad_series(dtw_train_x[1426:], window_size, multiplier)
ww_dtw_train_x = pd.concat([warped_train_x, padded_dtw], axis=0, ignore_index=True)
ww_dtw_train_y = np.append(warped_train_y, dtw_train_y[1426:], axis=0)
ww_dtw_test_x = padded_test_x
ww_dtw_test_y = test_y
print(ww_dtw_train_x.shape)
print(ww_dtw_train_y.shape)

(4274, 36)
(4274,)


Accuracy: 86.05042016806723

In [142]:
dtw_ww_tic = time.perf_counter()
ww_dtw_accuracy, ww_dtw_preds = rocket.run_classifier(ww_dtw_train_x, ww_dtw_train_y, ww_dtw_test_x, ww_dtw_test_y)
runtime = tic - time.perf_counter()
print(f"Accuracy: {ww_dtw_accuracy}\nTime Taken: {time.perf_counter() - dtw_ww_tic}")

[Pipeline] ............ (step 1 of 3) Processing rocket, total= 7.3min
[Pipeline] ......... (step 2 of 3) Processing normalise, total=   1.9s
[Pipeline] ............. (step 3 of 3) Processing model, total=  28.2s
Accuracy: 86.05042016806723
Time Taken: 532.7566456999921


<br><br>
#### DTW and Reverse

In [115]:
dtw_rev_train_x = pd.concat([train_x, dtw_train_x[1426:], reversed_train_x[1426:]], axis=0, ignore_index=True)
dtw_rev_train_y = np.append(train_y, dtw_train_y[1426:], axis=0)
dtw_rev_train_y = np.append(dtw_rev_train_y, reversed_train_y[1426:], axis=0)
dtw_rev_test_x = test_x
dtw_rev_test_y = test_y
print(dtw_rev_train_x.shape)
print(dtw_rev_train_y.shape)

(4274, 36)
(4274,)


Accuracy: 82.52100840336134

In [143]:
dtw_rev_tic = time.perf_counter()
dtw_rev_accuracy, dtw_rev_preds = rocket.run_classifier(dtw_rev_train_x, dtw_rev_train_y, dtw_rev_test_x, dtw_rev_test_y)
print(f"Accuracy: {dtw_rev_accuracy}\nTime Taken: {time.perf_counter() - dtw_rev_tic}")

[Pipeline] ............ (step 1 of 3) Processing rocket, total= 6.1min
[Pipeline] ......... (step 2 of 3) Processing normalise, total=   1.7s
[Pipeline] ............. (step 3 of 3) Processing model, total=  25.3s
Accuracy: 82.52100840336134
Time Taken: 442.42052130000957


<br><br>
#### Window Warp and Reverse

In [117]:
padded_rev = aug.pad_series(reversed_train_x[1426:], window_size, multiplier)
ww_rev_train_x = pd.concat([warped_train_x, padded_rev], axis=0, ignore_index=True)
ww_rev_train_y = np.append(warped_train_y, reversed_train_y[1426:], axis=0)
ww_rev_test_x = padded_test_x
ww_rev_test_y = test_y
print(ww_rev_train_x.shape)
print(ww_rev_train_y.shape)

(4278, 36)
(4278,)


Accuracy: 78.82352941176471

In [144]:
ww_rev_tic = time.perf_counter()
ww_rev_accuracy, ww_rev_preds = rocket.run_classifier(ww_rev_train_x, ww_rev_train_y, ww_rev_test_x, ww_rev_test_y)
print(f"Accuracy: {ww_rev_accuracy}\nTime Taken: {time.perf_counter() - ww_rev_tic}")

[Pipeline] ............ (step 1 of 3) Processing rocket, total= 7.3min
[Pipeline] ......... (step 2 of 3) Processing normalise, total=   1.9s
[Pipeline] ............. (step 3 of 3) Processing model, total=  26.4s
Accuracy: 78.82352941176471
Time Taken: 529.6813672999997


#### DTW and Window Warp and Reverse

In [119]:
padded_reversed = aug.pad_series(reversed_train_x[1426:], window_size, multiplier)
trio_train_x = pd.concat([warped_train_x, padded_dtw, padded_reversed], axis=0, ignore_index=True)
trio_train_y = np.append(ww_dtw_train_y, reversed_train_y[1426:], axis=0)
trio_test_x = padded_test_x
trio_test_y = test_y
print(trio_train_x.shape)
print(trio_train_y.shape)

(5700, 36)
(5700,)


Accuracy: 83.19327731092437

In [145]:
trio_tic = time.perf_counter()
trio_accuracy, trio_preds = rocket.run_classifier(trio_train_x, trio_train_y, trio_test_x, trio_test_y)
print(f"Accuracy: {trio_accuracy}\nTime Taken: {time.perf_counter() - trio_tic}")

[Pipeline] ............ (step 1 of 3) Processing rocket, total= 9.8min
[Pipeline] ......... (step 2 of 3) Processing normalise, total=   2.4s
[Pipeline] ............. (step 3 of 3) Processing model, total=  56.2s
Accuracy: 83.19327731092437
Time Taken: 705.7330419999926
